<a href="https://colab.research.google.com/github/ElahehJafarigol/Federated-Learning-GANs/blob/main/SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Deep Imbalanced Learning for Weather Data: A Federated Learning Approach**

In [ ]:
!pip install --quiet keras==2.9.0
!pip install --quiet tensorflow==2.9.2

In [ ]:
!pip install --quiet imbalanced-learn

In [ ]:
import os
from  IPython import display
import pathlib
import shutil
import tempfile
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')
from pandas.core.common import random_state
import pandas as pd
import numpy as np
import random
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

random_state = 42
random_seed = 42
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD
from keras import backend as K
from keras import datasets, layers, models, metrics
from keras.layers import GaussianNoise
from keras import regularizers

from keras.layers.serialization import activation
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.activation import ReLU
from pandas.core.indexes.datetimes import Resolution
from keras.layers import Concatenate

import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.metrics import roc_auc_score, confusion_matrix14

from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.activation import ReLU
from pandas.core.indexes.datetimes import Resolution

####**Multiple Datasets**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving WeatherAUS.csv to WeatherAUS.csv
User uploaded file "WeatherAUS.csv" with length 14159645 bytes


In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded['WeatherAUS.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
#X.head()

(142193, 24)

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(176505, 13)
(176505,)
(44127, 13)
(44127,)


#####Training Federated Learning

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
def create_clients(X_train, y_train, num_clients, initial='client'):
    # create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    # randomize the data before splitting the data between clients
    indices = np.random.permutation(len(X_train))
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]

    # shard data and place at each client
    size = len(X_train) // num_clients
    remainder = len(X_train) % num_clients
    shards = [(X_shuffled[i:i+size], y_shuffled[i:i+size]) for i in range(0, size*num_clients, size)]
    if remainder:
        shards[-1] = (np.concatenate([shards[-1][0], X_shuffled[-remainder:]]),
                      np.concatenate([shards[-1][1], y_shuffled[-remainder:]]))

    # number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    # create dictionary of clients and their shards
    clients = {}
    for i in range(len(client_names)):
        # create dictionary entry with client name and data shape
        clients[client_names[i]] = (shards[i][0].shape, shards[i][1].shape)

    return clients

def batch_data(data_shard, batchsize=Batch_size):
    # unpack data shard into data and labels arrays
    X, y = data_shard

    # create a tensorflow dataset object from the data and labels
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # shuffle and batch the dataset
    return dataset.shuffle(len(y)).batch(batchsize)

# create clients from training data and associated labels
X_train = X_train.values
y_train = y_train.values
clients = create_clients(X_train, y_train, num_clients=10, initial='client')
for client in clients:
    print(f"{client}: X={clients[client][0]}, y={clients[client][1]}")

# You can access the shards for each client by indexing the dictionary returned
# by create_clients(), like so:
client_1_shard = clients['client_1']
client_1_shard

# create a batched dataset for each client's data shard
clients_batched = {}
for client in clients:
    client_data_shard = (X_train, y_train)
    batched_data = batch_data(client_data_shard)
    clients_batched[client] = batched_data

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

# Federated Averaging: aggregation method
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the batch_size
    batch_size = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*batch_size
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*batch_size
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    # Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

def test_model(X_test, Y_test, model, communication_round):
    cce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    logits = model.predict(X_test, batch_size=Batch_size)
    logits = tf.squeeze(logits, axis=1) # remove last dimension from logits
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.round(logits), Y_test)
    auc = roc_auc_score(Y_test, logits)
    tn, fp, fn, tp = confusion_matrix(Y_test, tf.round(logits)).ravel()
    g_mean = np.sqrt(tp/(tp+fn)*tn/(tn+fp))
    print('communication_round: {} | global_accuracy: {:.3%} | global_loss: {} | global_AUC: {:.3%} | global_G-mean: {:.3%}'.format(communication_round, acc, loss, auc, g_mean))
    return acc, loss, auc, g_mean


In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = "BinaryAccuracy"

metrics = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

class MyModel:
    @staticmethod
    def build(shape, classes):
        model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(13,)),
        layers.GaussianNoise(stddev = 0.5),
        layers.Dense(128, activation='relu'),
        layers.GaussianNoise(stddev = 0.3),
        layers.Dense(64, activation='relu'),
        layers.GaussianNoise(stddev = 0.1),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
        ])
        return model


In [ ]:
smlp_global = MyModel()
global_model = smlp_global.build(shape=(13,), classes=2)
global_model.summary()

global_acc_list = []
global_loss_list = []
global_auc_list = []
global_gmean_list = []

# commence global training loop
for communication_round in range(communication_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    # initial list to collect local model weights after scaling
    scaled_local_weight_list = list()

    # randomize client data - using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # loop through each client and create new local model
    for client in client_names:
        smlp_local = MyModel()
        local_model = smlp_local.build(shape=(13,), classes=2)
        #local_model = smlp_local.build()
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)

        # set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        # fit local model with client's data
        model_history = local_model.fit(clients_batched[client], epochs=epochs, verbose=1)

        # evaluate local model on test data
        # test_loss, test_accuracy = local_model.evaluate(X_test, y_test)
        # print(f'Local model {client} - test_loss: {test_loss} - test_accuracy: {test_accuracy}')

        # scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # clear session to free memory after each communication round
        K.clear_session()

    # to get the average over all the local model, we simply take the sum of the scaled weights
    avg_grad = sum_scaled_weights(scaled_local_weight_list)

    # update global model
    global_model.set_weights(avg_grad)

    # test global model and print out metrics after each communications round
    for X_test, y_test in test_batched:
        global_acc, global_loss, global_auc, global_gmean = test_model(X_test, y_test, global_model, communication_round)
        global_acc_list.append(global_acc)
        global_loss_list.append(global_loss)
        global_auc_list.append(global_auc)
        global_gmean_list.append(global_gmean)


Streaming output truncated to the last 5000 lines.
Epoch 30/30
2758/2758 [==============================] - 13s 5ms/step - loss: 0.4337 - tp: 70262.0000 - fp: 18146.0000 - tn: 70176.0000 - fn: 17921.0000 - accuracy: 0.7957 - precision: 0.7947 - recall: 0.7968 - auc: 0.8800 - prc: 0.8805
Epoch 1/30
2758/2758 [==============================] - 17s 5ms/step - loss: 0.4558 - tp: 138552.0000 - fp: 36675.0000 - tn: 139969.0000 - fn: 37814.0000 - accuracy: 0.7890 - precision: 0.7907 - recall: 0.7856 - auc: 0.8732 - prc: 0.8745
Epoch 2/30
2758/2758 [==============================] - 14s 5ms/step - loss: 0.4542 - tp: 68459.0000 - fp: 18544.0000 - tn: 69778.0000 - fn: 19724.0000 - accuracy: 0.7832 - precision: 0.7869 - recall: 0.7763 - auc: 0.8671 - prc: 0.8694
Epoch 3/30
2758/2758 [==============================] - 14s 5ms/step - loss: 0.4522 - tp: 68542.0000 - fp: 18414.0000 - tn: 69908.0000 - fn: 19641.0000 - accuracy: 0.7844 - precision: 0.7882 - recall: 0.7773 - auc: 0.8684 - prc: 0.8706
Ep

####**Single Dataset**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded_1 = files.upload()

for fn in uploaded_1.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded_1[fn])))

In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded_1['Client1.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

In [ ]:
Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]
#Data.head()

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

Data.shape
Data.dropna(inplace=True)

# Separate the features and labels
X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

print("X:", X.shape)
print("y:", y.shape)

# Normalize the features
scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
X = pd.DataFrame(X.reset_index(drop=True))

original_indices = set(Data.index)
current_indices = set(X.index)

missing_indices = original_indices - current_indices
print(missing_indices)

X_single = X
y_single = y

#####Data Augmentation

In [ ]:
X = X_single
y = y_single

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"

metrics = [keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

model = tf.keras.Sequential([
layers.Dense(128, activation='relu', input_shape=(13,)),
layers.Dense(64, activation='relu'),
layers.Dense(32, activation='relu'),
layers.Dense(1, activation='sigmoid')
])


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

model.fit(X_train, y_train, epochs=Epochs, batch_size=Batch_size)

Epoch 1/30
440/440 [==============================] - 4s 3ms/step - loss: 0.5226 - tp: 10207.0000 - fp: 3529.0000 - tn: 10560.0000 - fn: 3860.0000 - accuracy: 0.7376 - precision: 0.7431 - recall: 0.7256 - auc: 0.8183 - prc: 0.8205
Epoch 2/30
440/440 [==============================] - 1s 2ms/step - loss: 0.4851 - tp: 10558.0000 - fp: 3139.0000 - tn: 10950.0000 - fn: 3509.0000 - accuracy: 0.7639 - precision: 0.7708 - recall: 0.7506 - auc: 0.8457 - prc: 0.8455
Epoch 3/30
440/440 [==============================] - 1s 2ms/step - loss: 0.4817 - tp: 10530.0000 - fp: 3104.0000 - tn: 10985.0000 - fn: 3537.0000 - accuracy: 0.7641 - precision: 0.7723 - recall: 0.7486 - auc: 0.8479 - prc: 0.8482
Epoch 4/30
440/440 [==============================] - 1s 2ms/step - loss: 0.4753 - tp: 10689.0000 - fp: 3183.0000 - tn: 10906.0000 - fn: 3378.0000 - accuracy: 0.7670 - precision: 0.7705 - recall: 0.7599 - auc: 0.8520 - prc: 0.8531
Epoch 5/30
440/440 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
Evaluation = model.evaluate(X_test, y_test)

220/220 [==============================] - 2s 2ms/step - loss: 0.4528 - tp: 2957.0000 - fp: 961.0000 - tn: 2548.0000 - fn: 574.0000 - accuracy: 0.7820 - precision: 0.7547 - recall: 0.8374 - auc: 0.8711 - prc: 0.8726
